<a href="https://colab.research.google.com/github/Tombellens/lipar-data-clean-analysis/blob/main/Copy_of_linkedin_cv_parsing_educations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install PyPDF2

import os
import pandas as pd
import json
import os
import requests
import subprocess
from PyPDF2 import PdfReader
from openai import OpenAI
import dateutil
client = OpenAI(
   api_key = #[openAIKey]
)

In [ ]:
def normalize_educations(json_data):
  print(json_data)
  normalized_data = []
  for education in json_data['education']:
    normalized_education = {
            "degree": "",
            "institution": "",
            "fieldOfStudy": "",
            "start-date": "",
            "end-date":""}
    normalized_education["degree"] = education.get("degree", "")
    normalized_education["institution"] = education.get("institution", "")
    normalized_education["fieldOfStudy"] = education.get("fieldOfStudy", "")
    normalized_education["start-date"] = education.get("startDate", "")
    normalized_education["end-date"] = education.get("endDate", "")

    normalized_data.append(normalized_education)

  return normalized_data


def get_education(linkedinCV):
      reader = PdfReader(linkedinCV)
      number_of_pages = len(reader.pages)
      full_text = ''
      for page in reader.pages:
         full_text = full_text + page.extract_text()

      print(full_text)
      response_json_education = client.chat.completions.create(
      model='gpt-4o-mini',
      response_format={"type": "json_object"},
      messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": "The user sends you a PDF resume in text format. You should parse this file and return information on their education in a JSON file in the following format:{\"education\":[{\"degree\": , \"institution\": ,\"fieldOfStudy\" \"startDate\", \"endDate\"}]}.The 'degree' field is exclusively meant for titles such as 'Master, PhD, Bachelor,.. ADHERE TO THIS FORMAT, THE KEYS AND ADD NO TEXT AS YOUR RESPONSE WILL BE USED BY A PARSER",},
      {"role": "user", "content": "This is the PDF text: " + full_text}])
      print(response_json_education.choices[0].message.content)
      try:
        return normalize_educations(json.loads(response_json_education.choices[0].message.content))

      except:
        print('Something went wrong while parsing attempt 1. Trying a second time..')
        response_json_education = client.chat.completions.create(
        model='gpt-4o-mini',
        response_format={"type": "json_object"},
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "The user has already sent you PDF resume in text format but will send you again in the next message.  You should parse this file and return information on their education in a JSON file in the following format:{\"education\":[{\"degree\": , \"institution\": ,\"fieldOfStudy\" \"startDate\", \"endDate\"}]}. The 'degree' field is exclusively meant for titles such as 'Master, PhD, Bachelor,.. PREVIOUSLY YOU MADE A MISTAKE SO ADHERE TO THIS FORMAT, THE KEYS AND ADD NO TEXT AS YOUR RESPONSE WILL BE USED BY A PARSER",},
        {"role": "user", "content": "This is the PDF text: " + full_text}])

        return normalize_educations(json.loads(response_json_education.choices[0].message.content))






In [ ]:
fileDirs = os.listdir('/content/drive/MyDrive/federal_json_entries_named/')
df_FL = pd.DataFrame()
linkedinCVs = os.listdir('/content/drive/MyDrive/linkedin_belgium')

trouble = [] #Ambiguous name-linkedin matches
trouble_parsing = []
already_parsed = os.listdir('/content/drive/MyDrive/linkedin_json_federal/education/')
for fileDir in fileDirs:
  person_positions = json.load(open('/content/drive/MyDrive/federal_json_entries_named/' + fileDir))
  print(person_positions)
  position = person_positions[0]
  if (position['first_name'] + "__" + position['last_name']) not in already_parsed:
    for linkedinCV in linkedinCVs:
      if position['first_name'].lower() in linkedinCV.lower() and position['last_name'].lower() in linkedinCV.lower():
        print('match' + position['first_name'] + " " + position['last_name'] + "-" + linkedinCV)
        try:
          positions = get_education('/content/drive/MyDrive/linkedin_belgium/' + linkedinCV)
          print(positions)
          json_object = json.dumps(positions, indent=4)
          print(json_object)
          if not os.path.exists('/content/drive/MyDrive/linkedin_json_federal/education/' + position['first_name'] + "__" + position['last_name'] + '/'):
                os.makedirs('/content/drive/MyDrive/linkedin_json_federal/education/' + position['first_name'] + "__" + position['last_name'] + '/')
          with open('/content/drive/MyDrive/linkedin_json_federal/education/' + position['first_name'] + "__" + position['last_name'] + '/' +  linkedinCV.replace('.pdf', '.json'), "w") as outfile:
            outfile.write(json_object)
        except:
          print('something went wrong. Adding to the list of problems')
          trouble_parsing.append(position['first_name'] + "-" + position['last_name'])




In [ ]:
len(os.listdir('/content/drive/MyDrive/linkedin_json_federal/education/'))

1909